In [ ]:
import numpy as np
import pandas

In [ ]:
!mkdir data/data_mean_teacher

In [ ]:
# UNLABELED DATA
# combine the encoded unlabeled data (split in many small files) in a single numpy array, which we subsequently write to a single file
path = "data/encoded_sentiment_data/"
unlabeledData = None
for i in range(0,103): # Change depending on how much data you want to process
    fileName = "embeddings_w266_"+str(i)+".npy"
    data = np.load(path+fileName)
    if unlabeledData is None:
        unlabeledData = data
    else:
        unlabeledData = np.concatenate([unlabeledData, data])
#     print(data.shape)
print(unlabeledData.shape)

In [ ]:
# write unlabeled data in a single file
unlabeledDataWithHeaders =np.zeros(unlabeledData.shape[0], dtype = [ ('x', np.float32, (500,)),('y', np.int32, ())])
unlabeledDataWithHeaders['x'] = unlabeledData
unlabeledDataWithHeaders['y'] = -1
print(type(unlabeledDataWithHeaders['x']))

finalFileUnlabeled = "data/data_mean_teacher/embeddings_w266_final.npy"
np.save(finalFileUnlabeled,unlabeledDataWithHeaders)

In [ ]:
# Check that it has been processed correctly
path = "data/data_mean_teacher/embeddings_w266_final.npy"
theData = np.load(path)
print(theData['x'].shape)
print(theData['y'][0])

In [ ]:
# LABELED DATA
# split labeled data into three separate files: test, dev, train (80, 10, 10)
# data needs to be in special form (see unlabeled processing above, x and y specifically)
# write out three files to the right location to be picked up by mean teacher


fileName = "data/encoded_data/embeddings_w266.npy"
encodedData = np.load(fileName)
encodedData = pandas.DataFrame(data = encodedData, columns = ["x_"+str(i) for i in range(500)])
filename = "data/Labeled_Colorado_Flu_Study_Tweets_AvI_RvN_SvO.csv"
coloradoData = pandas.read_csv(filename, sep="\t")
print(encodedData.shape)
allData = pandas.concat([coloradoData, encodedData], axis = 1)
print(allData.shape)

# get all data with labels present for the column we care about
allDataVal = allData.dropna(subset=["Related_Label"])
print(allDataVal.shape)

# extract X and Y as np arrays, so that we can feed them to tensors. 
x = allDataVal[["x_"+str(i) for i in range(500)]]
y = allDataVal["Related_Label"]
print(x.values.shape)
print(y.values.shape)

# split into train, test, dev (80%,10%,10%). Adjust the split ratio here
np.random.seed(42)
train_x_df, test_x_df, dev_x_df = np.split(x.sample(frac=1), [int(.8*len(x)), int(.9*len(x))])
train_y_df = y[train_x_df.index]
test_y_df = y[test_x_df.index]
dev_y_df = y[dev_x_df.index]

# convert to numpy arrays
train_x, test_x, dev_x, train_y, test_y, dev_y = train_x_df.values, test_x_df.values, dev_x_df.values, train_y_df.values, test_y_df.values, dev_y_df.values 
train_y = train_y.astype(int)
dev_y = dev_y.astype(int)
test_y = test_y.astype(int)
train_x = train_x.astype("float32")
test_x = test_x.astype("float32")
dev_x = dev_x.astype("float32")

In [ ]:
labeledDataTrain = np.zeros(train_x.shape[0], dtype = [ ('x', np.float32, (500,)),('y', np.int32, ())])
labeledDataTest = np.zeros(test_x.shape[0], dtype = [ ('x', np.float32, (500,)),('y', np.int32, ())])
labeledDataDev = np.zeros(dev_x.shape[0], dtype = [ ('x', np.float32, (500,)),('y', np.int32, ())])
labeledDataTrain['x'] = train_x
labeledDataTrain['y'] = train_y
labeledDataTest['x'] = test_x
labeledDataTest['y'] = test_y
labeledDataDev['x'] = dev_x
labeledDataDev['y'] = dev_y

In [ ]:
# write the actual data
finalFileLabeledTrain = "data/data_mean_teacher/encoded_labeled_train_data.npy"
np.save(finalFileLabeledTrain,labeledDataTrain)

finalFileLabeledTest = "data/data_mean_teacher/encoded_labeled_test_data.npy"
np.save(finalFileLabeledTest,labeledDataTest)

finalFileLabeledDev = "data/data_mean_teacher/encoded_labeled_dev_data.npy"
np.save(finalFileLabeledDev,labeledDataDev)